In [1]:
import open3d as o3d
import numpy as np
import os
import math

# 1) Filväg och utmapp
ply_path = r"C:\Users\lindh\Documents\GitHub\D7046E\Exercise1\filled_surface.ply"
out_dir  = r"C:/Users/lindh/Documents/GitHub/D7046E/Exercise1/Rendered_images"
os.makedirs(out_dir, exist_ok=True)

# 2) Läs in mesh och extrahera punktmoln
mesh = o3d.io.read_triangle_mesh(ply_path)
if not mesh.has_vertices():
    raise FileNotFoundError(f"Kan inte läsa mesh från {ply_path}")
pts = np.asarray(mesh.vertices)     # använder mesh-vertices som punktmoln

# 3) Sätt upp Visualizer
vis = o3d.visualization.Visualizer()
vis.create_window(width=1920, height=1080, visible=False)
vis.add_geometry(mesh)
opt = vis.get_render_option()
opt.mesh_show_back_face     = True
opt.mesh_shade_option       = o3d.visualization.MeshShadeOption.Color
opt.light_on                = True
view_ctl = vis.get_view_control()

# 4) Kamera-basparametrar
center      = mesh.get_center()
base_radius = 10
base_height = 100
height_var  = 5
num_views   = 15
angles      = np.linspace(0, 2*math.pi, num_views, endpoint=False)
zooms       = np.linspace(0.3, 0.4, num_views)
img_w, img_h = 1920, 1080  # bildstorlek för YOLO-normalisering

for i, (angle, zoom) in enumerate(zip(angles, zooms)):
    # — a) Horisontell rörelse
    x = base_radius * math.sin(angle)
    y = base_radius * math.cos(angle)
    # — b) Höjdvariation
    z = base_height + height_var * math.sin(angle*2)
    cam_pos = np.array([x, y, z])

    # — c1) Riktning mot objektets mitt
    front = center - cam_pos
    front /= np.linalg.norm(front)

    # — c2) Tilt: rotera front-vektorn 10° nedåt
    tilt_angle = math.radians(10)
    axis = np.cross(front, [0,0,1])
    axis /= np.linalg.norm(axis)
    K_mat = np.array([[0, -axis[2], axis[1]],
                      [axis[2], 0, -axis[0]],
                      [-axis[1], axis[0], 0]])
    R_mat = np.eye(3) + math.sin(tilt_angle)*K_mat + (1-math.cos(tilt_angle))*(K_mat @ K_mat)
    front_tilted = R_mat @ front

    # — d) Sätt kameran
    view_ctl.set_lookat(center.tolist())
    view_ctl.set_front(front_tilted.tolist())
    view_ctl.set_up([0, 0, 1])
    view_ctl.set_zoom(zoom)

    # >>> e) Hämta ut intrinsics & extrinsics <<<
    params = view_ctl.convert_to_pinhole_camera_parameters()
    K_mat   = params.intrinsic.intrinsic_matrix  # 3×3
    extr    = params.extrinsic                    # 4×4

    # — f) Projicera punkterna
    pts_h   = np.hstack((pts, np.ones((pts.shape[0],1))))
    cam_pts = (extr @ pts_h.T).T[:, :3]
    valid   = cam_pts[:,2] > 0
    cam_pts = cam_pts[valid]
    u       = (K_mat[0,0]*cam_pts[:,0]/cam_pts[:,2]) + K_mat[0,2]
    v       = (K_mat[1,1]*cam_pts[:,1]/cam_pts[:,2]) + K_mat[1,2]

    # — g) Bygg YOLO‐bbox (förutsätter att 'pts' redan filtrerats till bilpunkter)
    x_min, x_max = u.min(), u.max()
    y_min, y_max = v.min(), v.max()
    xc = ((x_min + x_max)/2) / img_w
    yc = ((y_min + y_max)/2) / img_h
    w  = (x_max - x_min) / img_w
    h  = (y_max - y_min) / img_h
    label = f"0 {xc:.5f} {yc:.5f} {w:.5f} {h:.5f}"

    # — h) Spara label + renderad bild
    txt_path = os.path.join(out_dir, f"view_{i:02d}.txt")
    with open(txt_path, "w") as f:
        f.write(label + "\n")

    img_path = os.path.join(out_dir, f"view_{i:02d}.png")
    vis.poll_events()
    vis.update_renderer()
    vis.capture_screen_image(img_path, do_render=True)
    print(f"Sparat vy {i}: {img_path}, label: {txt_path}")

# 5) Stäng fönstret
vis.destroy_window()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Sparat vy 0: C:/Users/lindh/Documents/GitHub/D7046E/Exercise1/Rendered_images\view_00.png, label: C:/Users/lindh/Documents/GitHub/D7046E/Exercise1/Rendered_images\view_00.txt
Sparat vy 1: C:/Users/lindh/Documents/GitHub/D7046E/Exercise1/Rendered_images\view_01.png, label: C:/Users/lindh/Documents/GitHub/D7046E/Exercise1/Rendered_images\view_01.txt
Sparat vy 2: C:/Users/lindh/Documents/GitHub/D7046E/Exercise1/Rendered_images\view_02.png, label: C:/Users/lindh/Documents/GitHub/D7046E/Exercise1/Rendered_images\view_02.txt
Sparat vy 3: C:/Users/lindh/Documents/GitHub/D7046E/Exercise1/Rendered_images\view_03.png, label: C:/Users/lindh/Documents/GitHub/D7046E/Exercise1/Rendered_images\view_03.txt
Sparat vy 4: C:/Users/lindh/Documents/GitHub/D7046E/Exercise1/Rendered_images\view_04.png, label: C:/Users/lindh/Documents/GitHub/D

In [2]:
import open3d as o3d
import numpy as np

# Ladda din point cloud
pcd = o3d.io.read_point_cloud("C:/Users/lindh/Documents/GitHub/D7046E/Exercise1/filled_surface.ply")
points = np.asarray(pcd.points)

# Kamera-egenskaper (exempel från din rendering)
K = np.array([[fx,  0, cx],
              [ 0, fy, cy],
              [ 0,  0,  1]])

# Extrinsic matrix (rotation + translation)
R = ...  # 3x3 rotation matrix
t = ...  # 3x1 translation vector
# Lägg till en kolumn med 1 för homogen koordinat
points_hom = np.hstack((points, np.ones((points.shape[0], 1))))

# Bygg extrinsic 4x4 matrix
extrinsic = np.eye(4)
extrinsic[:3, :3] = R
extrinsic[:3, 3] = t.flatten()

# Projicera till kamerans koordinatsystem
points_cam = (extrinsic @ points_hom.T).T[:, :3]

# Filtrera bort punkter bakom kameran
valid = points_cam[:, 2] > 0
points_cam = points_cam[valid]

# Projicera till 2D
u = (K[0, 0] * points_cam[:, 0] / points_cam[:, 2]) + K[0, 2]
v = (K[1, 1] * points_cam[:, 1] / points_cam[:, 2]) + K[1, 2]

def get_yolo_box(u, v, img_w, img_h):
    x_min, x_max = np.min(u), np.max(u)
    y_min, y_max = np.min(v), np.max(v)

    x_center = ((x_min + x_max) / 2) / img_w
    y_center = ((y_min + y_max) / 2) / img_h
    width = (x_max - x_min) / img_w
    height = (y_max - y_min) / img_h

    return f"0 {x_center:.5f} {y_center:.5f} {width:.5f} {height:.5f}"

label = get_yolo_box(u, v, img_w=640, img_h=480)

with open("Rendered_images/render_001.txt", "w") as f:
    f.write(label + "\n")


NameError: name 'fx' is not defined

In [ ]:
import open3d as o3d
import numpy as np

# 1) Ladda mesh
mesh = o3d.io.read_triangle_mesh("filled_surface.ply")
# (Hoppa över compute_vertex_normals om mesh redan har normala)

# 2) Skapa en Visualizer och lägg till meshen
vis = o3d.visualization.Visualizer()
vis.create_window(width=800, height=600)
vis.add_geometry(mesh)

# 3) (Valfritt) Rotera/vinkelställ kameran interaktivt
#    Kör denna rad, manipulera med musen i fönstret, och stäng med ESC:
vis.run()

# 4) Hämta ut kameraparametrar
ctr = vis.get_view_control()
cam_params = ctr.convert_to_pinhole_camera_parameters()

# 5) Intrinsics
K = cam_params.intrinsic.intrinsic_matrix
print("Intrinsics (K):\n", K)

# 6) Extrinsics
E = cam_params.extrinsic
print("Extrinsics (4×4):\n", E)

# 7) Städa upp
vis.destroy_window()